In [18]:
! pip install transformers==4.23.1
! pip install torch torchvision
! pip install tqdm

In [19]:
from transformers import AutoModel, AutoTokenizer
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import AdamW
import torch
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, TensorDataset

In [20]:
import torch
print("CUDA Available: ", torch.cuda.is_available())

CUDA Available:  True


In [21]:
torch.cuda.empty_cache()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [22]:
device

device(type='cuda', index=0)

In [23]:
file_path = 'Rationales_file_GAB_dataset_corrected.csv'
df = pd.read_csv(file_path)
train_df = df[df['exp_split'] == 'train']
test_df = df[df['exp_split'] == 'test']

In [25]:
print("Train df: ", len(train_df))
print("Test_df: ", len(test_df))

Train df:  7120
Test_df:  7120


In [26]:
import gc
# del variables
gc.collect()

18

In [27]:
model_name = 'GroNLP/HateBERT'
tokenizer = BertTokenizer.from_pretrained("GroNLP/hateBERT")
model = BertForSequenceClassification.from_pretrained("GroNLP/hateBERT")

Some weights of the model checkpoint at GroNLP/hateBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at GroNLP/hateBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should 

In [28]:
# model = model.cuda()

In [29]:
class CustomDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_length):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_length = max_length

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    texts = self.texts[idx]
    labels = self.labels[idx]
    encoding = self.tokenizer(texts, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
    # input_ids, mask_ids = torch.tensor(encoding['input_ids']), torch.tensor(encoding['attention_mask'])
    input_ids = encoding['input_ids'].squeeze()
    attention_mask = encoding['attention_mask'].squeeze()
    labels = labels
    return input_ids, attention_mask, labels

In [30]:
# Hyperparameters for tuning model initially. Let's see, we will change if required.
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [31]:
#Splitting training and validation testing split to test accuracy
train_text, val_texts, train_labels, val_labels = train_test_split(train_df['text'].tolist(),train_df['label'].tolist(), test_size = 0.2, random_state = 42)
train_dataset = CustomDataset(train_text, train_labels, tokenizer, max_length = 512)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer, max_length = 512)

#Creating dataloader object to train the model
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=True)

In [32]:
model = model.to(device)

In [33]:
import gc
# del variables
gc.collect()

284

In [42]:
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids, mask_ids, labels = batch
        if torch.cuda.is_available():
            input_ids = input_ids.to(device)
            mask_ids = mask_ids.to(device)
            labels = labels.to(device)
        # optimizer.grad()
        outputs = model(input_ids=input_ids, attention_mask=mask_ids, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Evaluating on Validation task
    model.eval()

    val_predictions = []
    val_labels = []

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, mask_ids, labels = batch
            if torch.cuda.is_available():
                input_ids = input_ids.to(device)
                mask_ids = mask_ids.to(device)
                labels = labels.to(device)
            outputs = model(input_ids=input_ids, attention_mask=mask_ids)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)
            val_predictions.extend(predictions.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(val_predictions, val_labels)
    print(f"Epoch {epoch + 1}: Validation Accuracy: {accuracy:.4f}")

Epoch 1: Validation Accuracy: 0.8315
Epoch 2: Validation Accuracy: 0.8315
Epoch 3: Validation Accuracy: 0.8315


In [ ]:
torch.save(model.state_dict(), '/content/gdrive/My Drive/fine_tuned_bert.pth')

In [ ]:
torch.save(model.state_dict(), 'fine_tuned_bert.pth')

In [ ]:
torch.save(model, 'fine_tuned_bert_Ayushi.pth')

# Testing on Tests Dataset for accuracy

In [43]:
test_texts = test_df['text'].tolist()
test_labels = test_df['label'].tolist()

In [44]:
test_dataset = CustomDataset(test_texts, test_labels, tokenizer, max_length = 512)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=True)

In [45]:
model.eval()
test_predictions = []
test_labels = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids, mask_ids, labels = batch
        if torch.cuda.is_available():
                input_ids = input_ids.to(device)
                mask_ids = mask_ids.to(device)
                labels = labels.to(device)
        outputs = model(input_ids=input_ids, attention_mask=mask_ids)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)
        test_predictions.extend(predictions.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(test_predictions, test_labels)

In [46]:
print("Accuracy of test dataset:", accuracy)

Accuracy of test dataset: 0.8370786516853933
